In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from geopy.distance import geodesic
from itertools import combinations
from tqdm.auto import tqdm
from icecream import ic
import random
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


# Lab 2 - Traveling Salesman Problem (TSP)

In [2]:
CITIES = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
num_cities = len(CITIES)
DIST_MATRIX = np.zeros((num_cities, num_cities))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Ancona,43.60,13.50
1,Andria,41.23,16.29
2,Bari,41.12,16.87
3,Bergamo,45.70,9.67
4,Bologna,44.50,11.34


In [3]:
# Fitness function to calculate the total distance of a TSP route
def tsp_cost(route):
    assert route[0] == route[-1], "Route should be a cycle"
    assert set(route) == set(range(len(CITIES))), "Route should visit all cities exactly once"
    total_cost = sum(DIST_MATRIX[route[i], route[i + 1]] for i in range(len(route) - 1))
    return total_cost


# Solution 1: Greedy Approximation
The greedy algorithm provides a fast, though approximate, solution:

1. Start from an initial city.
2. Select the nearest unvisited city as the next step.
3. Repeat until all CITIES are visited, then return to the start.

This approach does not guarantee an optimal solution but is fast and often produces a reasonably short route.

In [4]:
def greedy_tsp():
    distance_matrix = DIST_MATRIX.copy()
    visited = np.full(num_cities, False)
    city = 0 # Start from the first city (index 0)
    tsp = [city]  
    visited[city] = True
    total_distance = 0
    
    logging.debug(f"Starting at: {CITIES.at[city, 'name']}")
    
    while not np.all(visited):
        distance_matrix[:, city] = np.inf  # Set distances to already visited CITIES to infinity
        closest = np.argmin(distance_matrix[city])  # Find the closest unvisited city
        logging.debug(
            f"step: {CITIES.at[city, 'name']} -> {CITIES.at[closest, 'name']} "
            f"({distance_matrix[city, closest]:.2f}km)"
        )

        # Update route and mark city as visited        
        visited[closest] = True
        city = closest  # Move to the next city
        tsp.append(int(city))
    
    # Complete the cycle by returning to the starting city
    tsp.append(tsp[0])
    
    # Compute the total distance
    total_distance = tsp_cost(tsp)

    # Final log statement for the return step
    logging.debug(
        f"step: {CITIES.at[tsp[-2], 'name']} -> {CITIES.at[tsp[0], 'name']} "
        f"({distance_matrix[tsp[-2], tsp[0]]:.2f}km)"
    )
    logging.info(f"Result: Found a path of {len(tsp) - 1} steps, total length {total_distance:.2f}km")
    
    return tsp, total_distance


route, total_distance = greedy_tsp()
print("Greedy TSP Route:", route)
print("Total Distance:", total_distance)

2024-11-04 00:54:53,297 - INFO - Result: Found a path of 46 steps, total length 4436.03km


Greedy TSP Route: [0, 33, 12, 30, 9, 4, 19, 32, 25, 28, 18, 20, 3, 6, 44, 45, 23, 43, 41, 5, 40, 22, 42, 13, 16, 29, 10, 26, 39, 34, 15, 14, 21, 35, 11, 1, 2, 38, 17, 31, 8, 37, 24, 7, 36, 27, 0]
Total Distance: 4436.03176952516


# Solution 2 : Genetic Algorithm (GA)

The GA aims to provide a more accruate yet slower solution for the TSP problem. Here each individual (or chromosome) represents a potential route, and the goal is to evolve the population of routes to find shorter paths. To achieve a solution, the algorithm executes the following steps :

1. Initialization: Generate a population of random routes.
2. Fitness Calculation: Calculate the fitness of each route based on its total distance.
3. Selection: Select pairs of routes (parents) based on their fitness.
4. Crossover: Combine pairs of parents to produce offspring (new routes). Here PMX is chosen because it’s suited for permutation problems like TSP,     ensuring that offspring inherit a valid order of cities from both parents.
5. Mutation: Introduce random mutations to offspring to maintain genetic diversity. Here are the considered mutations :
    - Swap Mutation: Swaps two random cities in the route.
    - Scramble Mutation: Randomly shuffles a subset of cities in the route.
    - Inversion Mutation: Reverses a sequence of cities in the route. This is beneficial for TSP as it keeps cities in close proximity. 
6. Replacement: Replace the population with the new generation, keeping the best routes.

In [5]:
# Initialize population function with an optional pop_size parameter
def initialize_population(pop_size=None):
    # Default population size based on the number of cities
    if pop_size is None:
        if num_cities < 50:
            pop_size = 100
        elif num_cities < 200:
            pop_size = 200
        else:
            pop_size = 300

    population = []
    for _ in range(pop_size):
        # Generate a random route, then make it a valid cycle by setting the start and end points
        route = random.sample(range(1, num_cities), num_cities - 1)  # Exclude the start city
        route = [0] + route + [0]  # Start and end at city 0
        population.append(route)
    return population

def tournament_selection(population, tournament_size=5):
    # Randomly select individuals for the tournament
    tournament_contenders = random.sample(population, tournament_size)
    # Choose the individual with the best fitness (lowest distance) as the winner
    tournament_contenders = sorted(tournament_contenders, key=lambda tsp: tsp_cost(tsp))
    return tournament_contenders[0]

def swap_mutation(route):    
    mutated_route = route[:]    
    # Randomly select two different cities to swap
    i, j = random.sample(range(1, len(mutated_route) - 1), 2)  # Avoiding start and end city     
    # Swap the selected cities
    mutated_route[i], mutated_route[j] = mutated_route[j], mutated_route[i]    
    return mutated_route

def scramble_mutation(route):
    mutated_route = route[:]    
    # Randomly select two positions to define the scramble range
    start, end = sorted(random.sample(range(1, len(mutated_route) - 1), 2))  # Avoiding start and end if fixed    
    # Scramble (shuffle) the subset of cities within the range [start, end]
    subset = mutated_route[start:end + 1]
    random.shuffle(subset)
    mutated_route[start:end + 1] = subset
    
    return mutated_route

def inversion_mutation(route):
    mutated_route = route[:]    
    # Randomly select two positions to define the inversion range
    start, end = sorted(random.sample(range(1, len(mutated_route) - 1), 2))  # Avoiding start and end if fixed
    
    # Invert the subset of cities within the range [start, end]
    mutated_route[start:end + 1] = mutated_route[start:end + 1][::-1]
    
    return mutated_route
       
def pmx_crossover(parent1, parent2):
    # Initialize offspring with a placeholder for city indices  
    size = len(parent1)
    offspring = [-1] * size
    # Select crossover points randomly within the inner portion of the route
    start, end = sorted(random.sample(range(1, size - 1), 2))
    offspring[0] = offspring[-1] = 0  # Fix the start and end city

    # Copy the segment from the first parent to the offspring
    offspring[start:end + 1] = parent1[start:end + 1]    
    # Fill the remaining cities from parent2 while avoiding duplicates
    for i in range(start, end + 1):
        city = parent2[i]
        if city not in offspring:
            # Find where city from parent1 is mapped in parent2
            idx = i
            while offspring[idx] != -1:
                idx = parent2.index(parent1[idx])
            offspring[idx] = city

    # Fill in the remaining slots with cities from parent2
    for i in range(1, size - 1):  # Skip the fixed start and end positions
        if offspring[i] == -1:
            offspring[i] = parent2[i]

    if offspring[0] != offspring[-1]:
        logging.debug(
            f"Unvalid offspring: {offspring}"
        )
    assert offspring[0] == offspring[-1], "Unvalid TSP cycles"
    return offspring


    import random

def order_crossover(parent1, parent2):
    size = len(parent1)
    offspring = [-1] * size
    offspring[0] = offspring[-1] = parent1[0]  # Fix the start and end city (city 0)

    # Select two crossover points within the inner portion of the route
    start, end = sorted(random.sample(range(1, size - 1), 2))

    # Copy the segment from parent1 to offspring
    offspring[start:end + 1] = parent1[start:end + 1]

    # Fill the remaining positions with cities from parent2
    current_pos = end + 1
    for city in parent2:
        if city not in offspring:
            if current_pos >= size - 1:
                current_pos = 1  # Wrap around to the start after the fixed end
            offspring[current_pos] = city
            current_pos += 1

    # Validation to ensure the offspring is a valid TSP route
    if set(offspring) != set(parent1):
        logging.error(f"Invalid offspring generated in crossover: {offspring}")
        raise ValueError("Invalid offspring generated in crossover.")
    
    logging.debug(f"Order Crossover resulted in valid offspring: {offspring}")
    return offspring



In [7]:
# Full Genetic Algorithm with added logging for debugging
def genetic_algorithm_tsp(pop_size=None, generations=25, p_crossover=0.8, p_mutation=0.2):
    dist_matrix = DIST_MATRIX
    population = initialize_population(pop_size)
    
    logging.info(f"Initial population of size {len(population)} created.")

    best_solution = min(population, key=lambda tsp: tsp_cost(tsp))
    best_cost = tsp_cost(best_solution)
    logging.info(f"Initial best solution cost: {best_cost:.2f} km")

    for generation in range(generations):
        new_population = []

        # Elitism: Retain the top 10% solutions
        elite_count = max(1, int(0.1 * len(population)))
        sorted_population = sorted(population, key=lambda x: tsp_cost(x))
        new_population.extend(sorted_population[:elite_count])
        logging.debug(f"Generation {generation}: Retained {elite_count} elite individuals.")

        # Generate new offspring using crossover and mutation
        while len(new_population) < len(population):
            # Parent selection
            parent1 = tournament_selection(population)
            parent2 = tournament_selection(population)

            logging.debug(f"Selected parents for crossover: Parent1={parent1}, Parent2={parent2}")

            # Crossover
            if random.random() < p_crossover:
                offspring = order_crossover(parent1, parent2)
                logging.debug(f"Crossover occurred. Offspring generated: {offspring}")
            else:
                offspring = parent1[:]
                logging.debug(f"No crossover. Offspring is a copy of Parent1: {offspring}")

            # Mutation
            if random.random() < p_mutation:
                offspring = swap_mutation(offspring)
                logging.debug(f"Mutation occurred. Mutated offspring: {offspring}")

            # Add offspring to the new population
            new_population.append(offspring)
            logging.debug(f"Offspring added to new population. Current new population size: {len(new_population)}")

        # Update population and evaluate fitness for the next generation
        population = new_population
        current_best = min(population, key=lambda tsp: tsp_cost(tsp))
        current_best_cost = tsp_cost(current_best)

        # Update the best solution found so far
        if current_best_cost < best_cost:
            best_solution, best_cost = current_best, current_best_cost
            logging.info(f"Generation {generation}: New best cost found: {best_cost:.2f} km")

        # Log progress every 10 generations
        if generation % 10 == 0:
            logging.info(f"Generation {generation}: Best Cost = {best_cost:.2f} km")

    logging.info(f"Final best solution found with cost = {best_cost:.2f} km")
    return best_solution, best_cost

# Example usage (assuming CITIES and DIST_MATRIX are already defined)
best_route, best_distance = genetic_algorithm_tsp(generations=100)
print(f"Best route found: {best_route}, with a distance of {best_distance:.2f} km")

2024-11-04 00:55:35,832 - INFO - Initial population of size 100 created.
2024-11-04 00:55:35,847 - INFO - Initial best solution cost: 16847.87 km
2024-11-04 00:55:36,104 - INFO - Generation 0: New best cost found: 14174.25 km
2024-11-04 00:55:36,113 - INFO - Generation 0: Best Cost = 14174.25 km
2024-11-04 00:55:36,449 - INFO - Generation 2: New best cost found: 13242.92 km
2024-11-04 00:55:36,653 - INFO - Generation 3: New best cost found: 13190.55 km
2024-11-04 00:55:36,746 - INFO - Generation 4: New best cost found: 12969.74 km
2024-11-04 00:55:36,821 - INFO - Generation 5: New best cost found: 12499.19 km
2024-11-04 00:55:36,892 - INFO - Generation 6: New best cost found: 10892.63 km
2024-11-04 00:55:37,099 - INFO - Generation 9: New best cost found: 10811.66 km
2024-11-04 00:55:37,166 - INFO - Generation 10: New best cost found: 10693.03 km
2024-11-04 00:55:37,167 - INFO - Generation 10: Best Cost = 10693.03 km
2024-11-04 00:55:37,242 - INFO - Generation 11: New best cost found: 1

Best route found: [0, 33, 26, 27, 11, 1, 14, 15, 34, 39, 2, 38, 37, 8, 24, 35, 17, 31, 21, 36, 7, 42, 18, 25, 29, 41, 43, 44, 40, 5, 6, 22, 28, 20, 3, 13, 45, 23, 9, 30, 4, 19, 32, 16, 10, 12, 0], with a distance of 6933.95 km
